In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import sys
import scipy.io as io
from operator import add
sys.path.append('./model/')
utils = __import__('5275_Lab4_3')
model = __import__('model_pytorch')

In [2]:
subject_list = ['S01', 'S02', 'S03', 'S04', 'S05', 'S06', 'S07', 'S08', 'S09']
data_path = './BCICIV_2a_mat/'
dev = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
cls_criterion = nn.CrossEntropyLoss()
clf_epoch = 500
shuffle = True

# Part 1
## Problem 1

In [3]:
x_train, y_train, x_test, y_test = utils.load_data(data_path=data_path, subject='S01', dev=dev)
trainloader, testloader = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [6]:
eegnet = model.EEGNet().to(dev)
params = list(eegnet.parameters())
optimizer = optim.Adam(params,lr=1e-3)

In [7]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    eegnet.train()
    for x_train, y_train in trainloader:
        
        pred = eegnet(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
eegnet.eval()
acc = 0
clf_acc = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count = torch.unique(testloader.dataset[:][1], return_counts=True)[1]
confuse_m = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader:
    pred = eegnet(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc[pred.argmax()] += 1
acc /=len(testloader)
clf_acc = (clf_acc/cls_count).tolist()
confuse_m = confuse_m.tolist()
print('Done.')

Starting Training...
Done.


In [8]:
utils.print_metric(model='EEGNet', scheme='Individual', acc=acc, clf_acc=clf_acc, conf_m=confuse_m)

Accuracy of EEGNet: 0.826389
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.861111
1	0.944444
2	0.722222
3	0.777778
-----------------------------------
confusion matrix (prediction by Ground Truth):
62 4 0 1
9 68 2 0
1 0 52 15
0 0 18 56


## Problem 2

In [5]:
lr_list = [0.003, 0.001, 0.0003, 0.0001]
batch_list = [64, 32, 8, 2]

In [6]:
print('Start Training...')
for batch in batch_list:
    for lr in lr_list:
        cls_criterion = nn.CrossEntropyLoss()
        eegnet = model.EEGNet().to(dev)
        params = list(eegnet.parameters())
        optimizer = optim.Adam(params,lr=lr)                       
        for epoch in range(clf_epoch):
            eegnet.train()
            for x_train, y_train in trainloader:
                
                pred = eegnet(x_train)
                loss = cls_criterion(pred, y_train)
                
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

        """ Testing """
        eegnet.eval()
        acc = 0
        for x_test, y_test in testloader:
            pred = eegnet(x_test)
            pred_label = pred.argmax().long()
            if pred_label == y_test:
                acc += 1
        acc /=len(testloader)
        print('Batch= {}, lr= {}, Acc= {:5f}'.format(batch, lr, acc))

Starting Training...
Batch= 64, lr= 0.003, Acc= 0.774306
Batch= 64, lr= 0.001, Acc= 0.777778
Batch= 64, lr= 0.0003, Acc= 0.798611
Batch= 64, lr= 0.0001, Acc= 0.743056
Batch= 32, lr= 0.003, Acc= 0.791667
Batch= 32, lr= 0.001, Acc= 0.815972
Batch= 32, lr= 0.0003, Acc= 0.739583
Batch= 32, lr= 0.0001, Acc= 0.729167
Batch= 8, lr= 0.003, Acc= 0.802083
Batch= 8, lr= 0.001, Acc= 0.809028
Batch= 8, lr= 0.0003, Acc= 0.746528
Batch= 8, lr= 0.0001, Acc= 0.722222
Batch= 2, lr= 0.003, Acc= 0.739583
Batch= 2, lr= 0.001, Acc= 0.781250
Batch= 2, lr= 0.0003, Acc= 0.736111
Batch= 2, lr= 0.0001, Acc= 0.729167


## Problem 3
### Subject independent

In [9]:
x_train, y_train, x_test, y_test = utils.load_data_si(data_path=data_path,subject_list=subject_list,  subject2leave='S01', dev=dev)
trainloader_si, testloader_si = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [10]:
eegnet_si = model.EEGNet().to(dev)
params = list(eegnet_si.parameters())
optimizer = optim.Adam(params, lr=1e-3)

In [11]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    eegnet_si.train()
    for x_train, y_train in trainloader_si:
        
        pred = eegnet_si(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
eegnet_si.eval()
acc = 0
clf_acc_si = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count_si = torch.unique(testloader_si.dataset[:][1], return_counts=True)[1]
confuse_m_si = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader_si:
    pred = eegnet_si(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m_si[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc_si[pred.argmax()] += 1
acc /=len(testloader_si)
clf_acc_si = (clf_acc_si/cls_count_si).tolist()
confuse_m_si = confuse_m_si.tolist()
print('Done.')

Start Training...
Done.


In [12]:
utils.print_metric(model='EEGNet', scheme='SI', acc, clf_acc_si, confuse_m_si)

Accuracy of EEGNet: 0.635417
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.944444
1	0.750000
2	0.666667
3	0.180556
-----------------------------------
confusion matrix (prediction by Ground Truth):
68 18 20 21
4 54 1 1
0 0 48 37
0 0 3 13


### Subject dependent

In [3]:
x_train, y_train, x_test, y_test = utils.load_data_sd(data_path=data_path,subject_list=subject_list,  subject2leave='S01', dev=dev)
trainloader_sd, testloader_sd = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [4]:
eegnet_sd = model.EEGNet().to(dev)
params = list(eegnet_sd.parameters())
optimizer = optim.Adam(params, lr=1e-3)

In [5]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    eegnet_sd.train()
    for x_train, y_train in trainloader_sd:
        
        pred = eegnet_sd(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
eegnet_sd.eval()
acc = 0
clf_acc_sd = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count_sd = torch.unique(testloader_sd.dataset[:][1], return_counts=True)[1]
confuse_m_sd = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader_sd:
    pred = eegnet_sd(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m_sd[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc_sd[pred.argmax()] += 1
acc /=len(testloader_sd)
clf_acc_sd = (clf_acc_sd/cls_count_sd).tolist()
confuse_m_sd = confuse_m_sd.tolist()
print('Done.')

Start Training...
Done.


In [8]:
utils.print_metric(model='EEGNet', scheme='SD', acc=acc, clf_acc=clf_acc_sd, conf_m=confuse_m_sd)

Accuracy of EEGNetSD: 0.781250
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.930556
1	0.847222
2	0.861111
3	0.486111
-----------------------------------
Confusion matrix (Prediction by Ground Truth):
67 9 3 4
1 61 0 0
3 2 62 33
1 0 7 35


### SI+FT

In [13]:
x_train, y_train, x_test, y_test = utils.load_data(data_path=data_path, subject='S01', dev=dev)
trainloader_sift, testloader_sift = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [14]:
params = list(eegnet_si.parameters())
optimizer = optim.Adam(params, lr=1e-3)

In [15]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    eegnet_si.train()
    for x_train, y_train in trainloader_sift:
        
        pred = eegnet_si(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
eegnet_si.eval()
acc = 0
clf_acc_sift = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count_sift = torch.unique(testloader_sift.dataset[:][1], return_counts=True)[1]
confuse_m_sift = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader_sift:
    pred = eegnet_si(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m_sift[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc_sift[pred.argmax()] += 1
acc /= len(testloader_sift)
clf_acc_sift = (clf_acc_sift/cls_count_sift).tolist()
confuse_m_sift = confuse_m_sift.tolist()
print('Done.')

Start Training...
Done.


In [16]:
utils.print_metric(model='EEGNet', scheme='SI+FT', acc, clf_acc_sift, confuse_m_sift)

Accuracy of EEGNet: 0.833333
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.958333
1	0.902778
2	0.944444
3	0.527778
-----------------------------------
confusion matrix (prediction by Ground Truth):
69 7 1 1
1 65 1 0
2 0 68 33
0 0 2 38


# Part 2
## Problem 1
---
## ShallowConvNet
### Individual

In [9]:
x_train, y_train, x_test, y_test = utils.load_data(data_path=data_path, subject='S01', dev=dev)
trainloader, testloader = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [10]:
scnet = model.ShallowConvNet().to(dev)
params = list(scnet.parameters())
optimizer = optim.Adam(params,lr=1e-3)

In [11]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    scnet.train()
    for x_train, y_train in trainloader:
        
        pred = scnet(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
scnet.eval()
acc = 0
clf_acc = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count = torch.unique(testloader.dataset[:][1], return_counts=True)[1]
confuse_m = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader:
    pred = scnet(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc[pred.argmax()] += 1
acc /=len(testloader)
clf_acc = (clf_acc/cls_count).tolist()
confuse_m = confuse_m.tolist()
print('Done.')

Start Training...
Done.


In [12]:
utils.print_metric(model='ShallowConvNet', scheme='Individual', acc=acc, clf_acc=clf_acc, conf_m=confuse_m)

Accuracy of ShallowConvNetIndividual: 0.652778
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.722222
1	0.805556
2	0.472222
3	0.611111
-----------------------------------
Confusion matrix (Prediction by Ground Truth):
52 14 4 1
9 58 0 0
6 0 34 27
5 0 34 44


### Subject Independent

In [3]:
x_train, y_train, x_test, y_test = utils.load_data_si(data_path=data_path,subject_list=subject_list,  subject2leave='S01', dev=dev)
trainloader_si, testloader_si = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [4]:
scnet_si = model.ShallowConvNet().to(dev)
params = list(scnet_si.parameters())
optimizer = optim.Adam(params, lr=1e-3)

In [5]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    scnet_si.train()
    for x_train, y_train in trainloader_si:
        
        pred = scnet_si(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
scnet_si.eval()
acc = 0
clf_acc_si = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count_si = torch.unique(testloader_si.dataset[:][1], return_counts=True)[1]
confuse_m_si = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader_si:
    pred = scnet_si(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m_si[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc_si[pred.argmax()] += 1
acc /=len(testloader_si)
clf_acc_si = (clf_acc_si/cls_count_si).tolist()
confuse_m_si = confuse_m_si.tolist()
print('Done.')

Start Training...
Done.


In [6]:
utils.print_metric(model='ShallowConvNet', scheme='SI', acc=acc, clf_acc=clf_acc_si, conf_m=confuse_m_si)

Accuracy of ShallowConvNetSI: 0.642361
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.875000
1	0.819444
2	0.319444
3	0.555556
-----------------------------------
Confusion matrix (Prediction by Ground Truth):
63 9 17 11
8 59 10 7
0 1 23 14
1 3 22 40


### Subject dependent

In [7]:
x_train, y_train, x_test, y_test = utils.load_data_sd(data_path=data_path,subject_list=subject_list,  subject2leave='S01', dev=dev)
trainloader_sd, testloader_sd = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [8]:
scnet_sd = model.ShallowConvNet().to(dev)
params = list(scnet_sd.parameters())
optimizer = optim.Adam(params, lr=1e-3)

In [9]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    scnet_sd.train()
    for x_train, y_train in trainloader_sd:
        
        pred = scnet_sd(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
scnet_sd.eval()
acc = 0
clf_acc_sd = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count_sd = torch.unique(testloader_sd.dataset[:][1], return_counts=True)[1]
confuse_m_sd = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader_sd:
    pred = scnet_sd(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m_sd[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc_sd[pred.argmax()] += 1
acc /=len(testloader_sd)
clf_acc_sd = (clf_acc_sd/cls_count_sd).tolist()
confuse_m_sd = confuse_m_sd.tolist()
print('Done.')

Start Training...
Done.


In [10]:
utils.print_metric(model='ShallowConvNet', scheme='SD', acc=acc, clf_acc=clf_acc_sd, conf_m=confuse_m_sd)

Accuracy of ShallowConvNetSD: 0.750000
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.875000
1	0.944444
2	0.555556
3	0.625000
-----------------------------------
Confusion matrix (Prediction by Ground Truth):
63 3 6 6
7 68 3 1
1 0 40 20
1 1 23 45


### SI+FT

In [11]:
x_train, y_train, x_test, y_test = utils.load_data(data_path=data_path, subject='S01', dev=dev)
trainloader_sift, testloader_sift = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [12]:
params = list(scnet_si.parameters())
optimizer = optim.Adam(params, lr=1e-3)

In [13]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    scnet_si.train()
    for x_train, y_train in trainloader_sift:
        
        pred = scnet_si(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
scnet_si.eval()
acc = 0
clf_acc_sift = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count_sift = torch.unique(testloader_sift.dataset[:][1], return_counts=True)[1]
confuse_m_sift = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader_sift:
    pred = scnet_si(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m_sift[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc_sift[pred.argmax()] += 1
acc /= len(testloader_sift)
clf_acc_sift = (clf_acc_sift/cls_count_sift).tolist()
confuse_m_sift = confuse_m_sift.tolist()
print('Done.')

Start Training...
Done.


In [14]:
utils.print_metric(model='ShallowConvNet', scheme='SI+FT', acc=acc, clf_acc=clf_acc_sift, conf_m=confuse_m_sift)

Accuracy of ShallowConvNetSI+FT: 0.767361
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.861111
1	0.888889
2	0.541667
3	0.777778
-----------------------------------
Confusion matrix (Prediction by Ground Truth):
62 7 8 2
6 64 2 0
1 1 39 14
3 0 23 56


## SCCNet
### Individual

In [15]:
x_train, y_train, x_test, y_test = utils.load_data(data_path=data_path, subject='S01', dev=dev)
trainloader, testloader = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [16]:
sccnet = model.SCCNet().to(dev)
params = list(sccnet.parameters())
optimizer = optim.Adam(params,lr=1e-3)

In [17]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    sccnet.train()
    for x_train, y_train in trainloader:
        
        pred = sccnet(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
sccnet.eval()
acc = 0
clf_acc = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count = torch.unique(testloader.dataset[:][1], return_counts=True)[1]
confuse_m = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader:
    pred = sccnet(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc[pred.argmax()] += 1
acc /=len(testloader)
clf_acc = (clf_acc/cls_count).tolist()
confuse_m = confuse_m.tolist()
print('Done.')

Start Training...
Done.


In [18]:
utils.print_metric(model='SCCNet', scheme='Individual', acc=acc, clf_acc=clf_acc, conf_m=confuse_m)

Accuracy of SCCNetIndividual: 0.815972
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.958333
1	0.916667
2	0.583333
3	0.805556
-----------------------------------
Confusion matrix (Prediction by Ground Truth):
69 6 2 1
1 66 1 0
1 0 42 13
1 0 27 58


### Subject Independent

In [19]:
x_train, y_train, x_test, y_test = utils.load_data_si(data_path=data_path,subject_list=subject_list,  subject2leave='S01', dev=dev)
trainloader_si, testloader_si = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [20]:
sccnet_si = model.ShallowConvNet().to(dev)
params = list(sccnet_si.parameters())
optimizer = optim.Adam(params, lr=1e-3)

In [21]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    sccnet_si.train()
    for x_train, y_train in trainloader_si:
        
        pred = sccnet_si(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
sccnet_si.eval()
acc = 0
clf_acc_si = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count_si = torch.unique(testloader_si.dataset[:][1], return_counts=True)[1]
confuse_m_si = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader_si:
    pred = sccnet_si(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m_si[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc_si[pred.argmax()] += 1
acc /=len(testloader_si)
clf_acc_si = (clf_acc_si/cls_count_si).tolist()
confuse_m_si = confuse_m_si.tolist()
print('Done.')

Start Training...
Done.


In [22]:
utils.print_metric(model='SCCNet', scheme='SI', acc=acc, clf_acc=clf_acc_si, conf_m=confuse_m_si)

Accuracy of SCCNetSI: 0.680556
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.791667
1	0.777778
2	0.472222
3	0.680556
-----------------------------------
Confusion matrix (Prediction by Ground Truth):
57 11 9 9
8 56 6 3
5 3 34 11
2 2 23 49


### Subject dependent

In [23]:
x_train, y_train, x_test, y_test = utils.load_data_sd(data_path=data_path,subject_list=subject_list,  subject2leave='S01', dev=dev)
trainloader_sd, testloader_sd = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [24]:
sccnet_sd = model.ShallowConvNet().to(dev)
params = list(sccnet_sd.parameters())
optimizer = optim.Adam(params, lr=1e-3)

In [25]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    scnet_sd.train()
    for x_train, y_train in trainloader_sd:
        
        pred = scnet_sd(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
scnet_sd.eval()
acc = 0
clf_acc_sd = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count_sd = torch.unique(testloader_sd.dataset[:][1], return_counts=True)[1]
confuse_m_sd = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader_sd:
    pred = scnet_sd(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m_sd[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc_sd[pred.argmax()] += 1
acc /=len(testloader_sd)
clf_acc_sd = (clf_acc_sd/cls_count_sd).tolist()
confuse_m_sd = confuse_m_sd.tolist()
print('Done.')

Start Training...
Done.


In [26]:
utils.print_metric(model='SCCNet', scheme='SD', acc=acc, clf_acc=clf_acc_sd, conf_m=confuse_m_sd)

Accuracy of SCCNetSD: 0.763889
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.875000
1	0.930556
2	0.486111
3	0.763889
-----------------------------------
Confusion matrix (Prediction by Ground Truth):
63 4 5 4
5 67 3 0
1 0 35 13
3 1 29 55


### SI+FT

In [27]:
x_train, y_train, x_test, y_test = utils.load_data(data_path=data_path, subject='S01', dev=dev)
trainloader_sift, testloader_sift = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [28]:
params = list(sccnet_si.parameters())
optimizer = optim.Adam(params, lr=1e-3)

In [29]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    sccnet_si.train()
    for x_train, y_train in trainloader_sift:
        
        pred = sccnet_si(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
sccnet_si.eval()
acc = 0
clf_acc_sift = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count_sift = torch.unique(testloader_sift.dataset[:][1], return_counts=True)[1]
confuse_m_sift = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader_sift:
    pred = sccnet_si(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m_sift[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc_sift[pred.argmax()] += 1
acc /= len(testloader_sift)
clf_acc_sift = (clf_acc_sift/cls_count_sift).tolist()
confuse_m_sift = confuse_m_sift.tolist()
print('Done.')

Start Training...
Done.


In [30]:
utils.print_metric(model='SCCNet', scheme='SI+FT', acc=acc, clf_acc=clf_acc_sift, conf_m=confuse_m_sift)

Accuracy of SCCNetSI+FT: 0.756944
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.958333
1	0.791667
2	0.597222
3	0.680556
-----------------------------------
Confusion matrix (Prediction by Ground Truth):
69 13 8 5
1 57 1 0
2 1 43 18
0 1 20 49


## TSception
### Individual

In [3]:
x_train, y_train, x_test, y_test = utils.load_data(data_path=data_path, subject='S01', dev=dev)
trainloader, testloader = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [7]:
TSception = model.TSception(num_classes=4, input_size=(1,22,562), sampling_rate=125, num_T=9, num_S=6, hidden=128, dropout_rate=0.3).to(dev)
params = list(TSception.parameters())
optimizer = optim.Adam(params,lr=1e-3)

In [8]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    TSception.train()
    for x_train, y_train in trainloader:
        
        pred = TSception(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
TSception.eval()
acc = 0
clf_acc = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count = torch.unique(testloader.dataset[:][1], return_counts=True)[1]
confuse_m = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader:
    pred = TSception(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc[pred.argmax()] += 1
acc /=len(testloader)
clf_acc = (clf_acc/cls_count).tolist()
confuse_m = confuse_m.tolist()
print('Done.')

Start Training...
Done.


In [9]:
utils.print_metric(model='TSception', scheme='Individual', acc=acc, clf_acc=clf_acc, conf_m=confuse_m)

Accuracy of TSceptionIndividual: 0.486111
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.347222
1	0.652778
2	0.333333
3	0.611111
-----------------------------------
Confusion matrix (Prediction by Ground Truth):
25 21 10 6
32 47 8 3
7 3 24 19
8 1 30 44


### Subject Independent

In [11]:
x_train, y_train, x_test, y_test = utils.load_data_si(data_path=data_path,subject_list=subject_list,  subject2leave='S01', dev=dev)
trainloader_si, testloader_si = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [10]:
TSception_si = model.TSception(num_classes=4, input_size=(1,22,562), sampling_rate=125, num_T=9, num_S=6, hidden=128, dropout_rate=0.3).to(dev)
params = list(TSception_si.parameters())
optimizer = optim.Adam(params,lr=1e-3)

In [12]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    TSception_si.train()
    for x_train, y_train in trainloader_si:
        
        pred = TSception_si(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
TSception_si.eval()
acc = 0
clf_acc_si = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count_si = torch.unique(testloader_si.dataset[:][1], return_counts=True)[1]
confuse_m_si = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader_si:
    pred = TSception_si(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m_si[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc_si[pred.argmax()] += 1
acc /=len(testloader_si)
clf_acc_si = (clf_acc_si/cls_count_si).tolist()
confuse_m_si = confuse_m_si.tolist()
print('Done.')

Start Training...
Done.


In [13]:
utils.print_metric(model='TSception', scheme='SI', acc=acc, clf_acc=clf_acc_si, conf_m=confuse_m_si)

Accuracy of TSceptionSI: 0.409722
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.555556
1	0.388889
2	0.250000
3	0.444444
-----------------------------------
Confusion matrix (Prediction by Ground Truth):
40 27 14 13
14 28 20 12
8 11 18 15
10 6 20 32


### Subject Dependent

In [14]:
x_train, y_train, x_test, y_test = utils.load_data_sd(data_path=data_path,subject_list=subject_list,  subject2leave='S01', dev=dev)
trainloader_sd, testloader_sd = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [15]:
TSception_sd = model.ShallowConvNet().to(dev)
params = list(TSception_sd.parameters())
optimizer = optim.Adam(params, lr=1e-3)

In [16]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    TSception_sd.train()
    for x_train, y_train in trainloader_sd:
        
        pred = TSception_sd(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
TSception_sd.eval()
acc = 0
clf_acc_sd = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count_sd = torch.unique(testloader_sd.dataset[:][1], return_counts=True)[1]
confuse_m_sd = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader_sd:
    pred = TSception_sd(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m_sd[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc_sd[pred.argmax()] += 1
acc /=len(testloader_sd)
clf_acc_sd = (clf_acc_sd/cls_count_sd).tolist()
confuse_m_sd = confuse_m_sd.tolist()
print('Done.')

Start Training...
Done.


In [17]:
utils.print_metric(model='TSception', scheme='SD', acc=acc, clf_acc=clf_acc_sd, conf_m=confuse_m_sd)

Accuracy of TSceptionSD: 0.750000
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.930556
1	0.819444
2	0.638889
3	0.611111
-----------------------------------
Confusion matrix (Prediction by Ground Truth):
67 9 6 6
4 59 3 4
1 2 46 18
0 2 17 44


### SI+FT

In [18]:
x_train, y_train, x_test, y_test = utils.load_data(data_path=data_path, subject='S01', dev=dev)
trainloader_sift, testloader_sift = utils.bci2a_dataloader(x_train, y_train, x_test, y_test,                                                              batch_size=32, shuffle_train=shuffle)

In [19]:
params = list(TSception_si.parameters())
optimizer = optim.Adam(params, lr=1e-3)

In [20]:
""" Training """
print('Start Training...')
for epoch in range(clf_epoch):
    TSception_si.train()
    for x_train, y_train in trainloader_sift:
        
        pred = TSception_si(x_train)
        loss = cls_criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

""" Testing """
TSception_si.eval()
acc = 0
clf_acc_sift = torch.tensor([0,0,0,0], dtype=torch.float).to(dev)
cls_count_sift = torch.unique(testloader_sift.dataset[:][1], return_counts=True)[1]
confuse_m_sift = torch.zeros((4,4), dtype=torch.int) # row = prediction, column = ground truth
for x_test, y_test in testloader_sift:
    pred = TSception_si(x_test)
    pred_label = pred.argmax().long()
    y_test = y_test.long()
    confuse_m_sift[pred_label, y_test] += 1
    if pred_label == y_test:
        acc += 1
        clf_acc_sift[pred.argmax()] += 1
acc /= len(testloader_sift)
clf_acc_sift = (clf_acc_sift/cls_count_sift).tolist()
confuse_m_sift = confuse_m_sift.tolist()
print('Done.')

Start Training...
Done.


In [21]:
utils.print_metric(model='TSception', scheme='SI+FT', acc=acc, clf_acc=clf_acc_sift, conf_m=confuse_m_sift)

Accuracy of TSceptionSI+FT: 0.538194
-----------------------------------
Overall Accuracy:
Label	Accuracy
0	0.416667
1	0.666667
2	0.388889
3	0.680556
-----------------------------------
Confusion matrix (Prediction by Ground Truth):
30 13 7 4
29 48 11 2
6 8 28 17
7 3 26 49
